In [23]:
%pip install jupyter_dash
import pandas as pd
import numpy as np
import os
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
os.chdir('../population_uk/')
import plotly.graph_objs as go

sheets_dict = pd.read_excel('datadownload.xls', sheet_name=None)
import itertools
app = JupyterDash(__name__)
region_options = sheets_dict['1998']['geogname'].unique()
app.layout = html.Div([
    html.H2("The age and sex structure of a population varies by local area"),
    html.Div([dcc.Dropdown(id = "Region",
                           options =[{
                               'label': i,
                               'value':i,
                           } for i in region_options],
                           value = "UNITED KINGDOM"),],
             style = {'width': "25%",
                      'display': 'inline-block'}),
            #dcc.Graph(id='pop-graph'),
    html.Div([dcc.Dropdown(id = "Year",
                       options =[{
                           'label': i,
                           'value':i,
                       } for i in list(sheets_dict.keys())],
                       value = "1998"),],
         style = {'width': "25%",
                  'display': 'inline-block'}),
        dcc.Graph(id='pop-graph')],
)


@app.callback(
    Output('pop-graph', 'figure'),
    #dash.dependencies.Output('pop-graph1', 'figure'),
    [Input('Region', 'value'), Input('Year', 'value')])

def update_graph(region, year):
    
    df = sheets_dict[year].copy()
    p = df.pop('geogcode')
    p = df.pop('f_tot')
    p = df.pop('m_tot')
    local_data = df[df['geogname'] == region]
    women_bins = local_data.loc[:,["f_" in i for i  in local_data.columns]].values[0] * -1#np.array([-600, -623, -653, -650, -670, -578, -541, -411, -322, -230])
    men_bins = local_data.loc[:,["m_" in i for i  in local_data.columns]].values[0]#np.array([600, 623, 653, 650, 670, 578, 541, 360, 312, 170])
    y = list(range(0, 99, 1))
    a = round(np.mean(men_bins), -3) * 1.5
    a = round(max([i for i in men_bins if str(i) != 'nan']), -3) * 2
    round(max(women_bins), -3) * 1.5 / 10
    b = a / 10
    right = [round(i, -2) for i in range(0, int(a), int(b))]
    left = [round(-i, -2) for i in range(0, int(a), int(b))]
    left.reverse()
    
    trace_list = [go.Bar(
  
               y=y,
               x=men_bins,
               orientation='h',
               name='Men',
               text= men_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='powderblue')
               ),
        go.Bar(y=y,
               x=women_bins,
               orientation='h',
               name='Women',
               text=-1 * women_bins.astype('int'),
               hoverinfo='text',
               marker=dict(color='green')
               )]
    layout_tmp = go.Layout(yaxis=go.layout.YAxis(title='Age'),
     xaxis=go.layout.XAxis(
                       range=[-round(max(men_bins), -3) * 1.5, round(max(men_bins), -3)*1.5],
                        tickvals =[i for  i in list(itertools.chain.from_iterable([left,right[1:]]))],
                       ticktext=list(itertools.chain.from_iterable([[-i for i in left] ,right[1:]])),
                        title='Number'),
                    barmode='overlay',
                  bargap=0.1)


    return {
        'data': trace_list,
        'layout': go.Layout(yaxis=go.layout.YAxis(title='Age'),
                   xaxis=go.layout.XAxis(
                       range=[-round(max(men_bins), -3) * 2, round(max(men_bins), -3)*2],
                       tickvals =[i for  i in list(itertools.chain.from_iterable([left,right[1:]]))],
                       ticktext=list(itertools.chain.from_iterable([[-i for i in left] ,right[1:]])),
                       title='Number'),
                   barmode='overlay',
                   bargap=0.1)
    }
app.run_server(mode='inline')